# Exercises

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

In [1]:
# establishing environment 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## Exercise 1-1

### Read the case, department, and source data into their own spark dataframes.

In [2]:
# reading in data and saving to separate DFs
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

In [3]:
# previewing source data
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [4]:
# previewing case data
case.show(5)

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222| 

In [5]:
# previewing dept data
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



## Exercise 1-2a

### Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [6]:
# storing source data in json format
source.write.json("sources_json", mode="overwrite")

In [7]:
# storing source data in csv format
source.write.csv("sources_csv", mode="overwrite")

## Exercise 1-2b
### Inspect your folder structure. What do you notice?

There are two files in each folder. The main file containing the data written from the DF and a separate success file.

## Exercise 1-3
### Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [10]:
# displaying data types of source DF columns
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [11]:
# displaying data types of case DF columns
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [12]:
# displaying data types of department DF columns
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [13]:
# converting case_closed and case_late from strings to booleans
# this will allow us to perform boolean operations on these columns should we need to in the future
case = case.withColumn("case_closed", expr('case_closed == "YES"'))\
.withColumn("case_late", expr('case_late == "YES"'))

In [15]:
# setting date and time format
fmt = "M/d/yy H:mm"

# converting dates to date time format
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

## Exercise 2-1a
### How old is the latest (in terms of days past SLA) currently open issue? 

In [16]:
# filtering by open cases (ie. case_closed == NO)
# displaying max number of days late (ie. past SLA)
(case.filter(case.case_closed == 'NO').select(max(case.num_days_late)).show())

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



## Exercise 2-1b
### How long has the oldest (in terms of days since opened) currently opened issue been open?

In [17]:
# adding case_age column that reflects how long an issue has been open 
# column shows difference in days between open data and current date
case = case.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))

# filtering for cases that are still open and finding max case age
(case.filter(case.case_closed == 'NO').select(max(case.case_age)).show())

+-------------+
|max(case_age)|
+-------------+
|         1430|
+-------------+



## Exercise 2-2
### How many Stray Animal cases are there?

In [18]:
# filtering for stray animal cases and using count to count number of rows
case.filter(case.service_request_type == 'Stray Animal').count()

26760

## Exercise 2-3
### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [19]:
# filtering for cases with a service request type that is not office standby
# filtering for cases with field operations set as dept_division
# counting number of rows that meet both filter criterions
case.filter(case.service_request_type != 'Officer Standby').filter(case.dept_division == 'Field Operations').count()

113902

## Exercise 2-4

### Convert the council_district column to a string column.

In [20]:
# converting council_district to string since its an id and thus we won't be performing mathematical operations 
# on it
case = case.withColumn("council_district", col("council_district").cast("string"))

## Exercise 2-5
### Extract the year from the case_closed_date column.

In [21]:
# extracting year from case_closed_date column
# displaying alongside case_id
case.select(year("case_closed_date").alias('year'), 'case_id').show()

+----+----------+
|year|   case_id|
+----+----------+
|2018|1014127332|
|2018|1014127333|
|2018|1014127334|
|2018|1014127335|
|2018|1014127336|
|2018|1014127337|
|2018|1014127338|
|2018|1014127339|
|2018|1014127340|
|2018|1014127341|
|2018|1014127342|
|2018|1014127343|
|2018|1014127344|
|2018|1014127345|
|2018|1014127346|
|2018|1014127347|
|2018|1014127348|
|2018|1014127349|
|2018|1014127350|
|2018|1014127351|
+----+----------+
only showing top 20 rows



## Exercise 2-6
### Convert num_days_late from days to hours in new columns num_hours_late.

In [22]:
# creating column "num_hours_late" that reflects number of hours past SLA
case = case.withColumn("num_hours_late", expr("num_days_late * 24 AS num_hours_late"))

# previewing column next to original column, num_days_late
case.select("num_days_late", "num_hours_late").show(5)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



## Exercise 2-7
### Join the case data with the source and department data.

In [23]:
case = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

# previewing merged DF vertically 
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_due_date        | 2018-01-01 00:42:00  
 case_age             | 1065                 
 num_hours_late       | -23964.2102784       
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [24]:
# left join on source_id 
case = (case.join(source, "source_id", "left"))

# previewing merged DF vertically 
case.show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_due_date        | 2018-01-01 00:42:00  
 case_age             | 1065                 
 num_hours_late       | -23964.2102784       
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS             
only showing top 1 row



## Exercise 2-8
### Are there any cases that do not have a request source?

In [25]:
# using isnull and count to count number of cases with null value as source_username
case.where(col("source_username").isNull()).count()

0

There are no cases with null source_usernames

## Exercise 2-9
### What are the top 10 service request types in terms of number of requests?

In [27]:
# grouping by service request type and counting number of rows per service type request
# displaying top 10 with highest number of requests
case.groupby('service_request_type').agg((count(case.service_request_type)).alias('number_of_requests'))\
.sort(desc('number_of_requests')).show(10)

+--------------------+------------------+
|service_request_type|number_of_requests|
+--------------------+------------------+
|           No Pickup|             89210|
|Overgrown Yard/Trash|             66403|
|        Bandit Signs|             32968|
|        Damaged Cart|             31163|
|Front Or Side Yar...|             28920|
|        Stray Animal|             27361|
|Aggressive Animal...|             25492|
|Cart Exchange Req...|             22608|
|Junk Vehicle On P...|             21649|
|     Pot Hole Repair|             20827|
+--------------------+------------------+
only showing top 10 rows



## Exercise 2-10
### What are the top 10 service request types in terms of average days late?

In [28]:
# grouping by service request type and calculating average days late 
# displaying top 10 with highest average days late
case.groupby('service_request_type').agg((mean(case.num_days_late)).alias('avg_days_late'))\
.sort(desc('avg_days_late')).show(10)


+--------------------+------------------+
|service_request_type|     avg_days_late|
+--------------------+------------------+
|  Zoning: Junk Yards| 175.9563621042095|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|     142.112556415|
|Zoning: Recycle Yard|135.92851612479797|
|Donation Containe...|131.75610506358706|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint| 72.87050230311695|
+--------------------+------------------+
only showing top 10 rows



## Exercise 2-11
### Does number of days late depend on department?

In [31]:
(
    case.filter('case_late')
    .groupby('department')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+------------------------+---------+------------+
|department              |days_late|n_cases_late|
+------------------------+---------+------------+
|Metro Health            |6.5      |854         |
|Solid Waste             |7.1      |33729       |
|Trans & Cap Improvements|10.7     |5529        |
|Parks & Recreation      |22.4     |3810        |
|Animal Care Services    |23.4     |23751       |
|DSD/Code Enforcement    |49.5     |26439       |
|Customer Service        |88.2     |2035        |
+------------------------+---------+------------+



## Exercise 2-12
### How do number of days late depend on department and request type?

In [33]:
(
    case.filter("case_closed")
    .filter("case_late")
    .groupby("department", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(desc("days_late"))
    .show(40, truncate=False)
)

+--------------------+--------------------------------------------+---------+-------+
|department          |service_request_type                        |days_late|n_cases|
+--------------------+--------------------------------------------+---------+-------+
|DSD/Code Enforcement|Zoning: Recycle Yard                        |273.6    |75     |
|DSD/Code Enforcement|Zoning: Junk Yards                          |251.9    |146    |
|DSD/Code Enforcement|Donation Container Enforcement              |201.7    |82     |
|DSD/Code Enforcement|Structure/Housing Maintenance               |182.4    |30     |
|DSD/Code Enforcement|Graffiti: Private Property (Corridors)      |175.1    |3      |
|DSD/Code Enforcement|Storage of Used Mattress                    |164.0    |7      |
|DSD/Code Enforcement|Labeling for Used Mattress                  |162.4    |7      |
|DSD/Code Enforcement|Record Keeping of Used Mattresses           |154.0    |7      |
|DSD/Code Enforcement|Signage Requied for Sale of Used